In [29]:
import socket
import re

class HTTPRequestBuilder:
    def __init__(self):
        self.request = ""
        self.method = "GET"
        self.path = "/"
        self.version = "HTTP/1.1"
        self.headers = {"Host":"example.com"}
        self.response_byte_length = 1024
        self.port = 80
        self.body = ""
        self.build()

    def build(self) -> HTTPRequestBuilder:
        self.request = (f"{self.method} {self.path} {self.version}\r\n")
        for key in self.headers:
            self.request +=  f"{key}: {self.headers[key]}\r\n"
        self.request += "\r\n"
        self.request += self.body
        return self

    def set_method(self, method:str) -> HTTPRequestBuilder:
        if method.upper() not in ["GET","POST","PUT","DELETE"]:
            raise Exception("Invalid Method")
        self.method = method.upper()
        self.build()
        return self
    
    def set_header(self, key:str, value:str) -> HTTPRequestBuilder:
        self.headers[key] = value
        self.build()
        return self

    def set_response_byte_length(self, len:int) -> HTTPRequestBuilder:
        self.response_byte_length = len
        return self

    def set_body(self, body:str) -> HTTPRequestBuilder:
        self.body = body
        self.build()
        return self

    def set_path(self, path:str) -> HTTPRequestBuilder:
        regex = re.compile(r'^\/[^\s?#]*(\?[^\s#]*)?(#[^\s]*)?$')
        if not regex.match(path):
            raise Exception("Invalid Path")
        self.path = path
        self.build()
        return self
    
    def set_version(self, version:str) -> HTTPRequestBuilder:
        regex = re.compile(r'^HTTP\/(1\.0|1\.1|2|3)$')
        if not regex.match(version):
            raise Exception("Invalid HTTP Version")
        self.version = version
        self.build()
        return self
    
    def send(self):
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((self.headers["Host"],self.port))
        sock.send(self.request.encode())
        return sock.recv(self.response_byte_length)
    
    def print(self):
        print(self.request)



In [32]:
request = HTTPRequestBuilder()
request.set_method("GET").set_header("Host","example.com").set_header("Connection","close").set_body("anything")
request.set_version("HTTP/1.1").set_path("/")
request.print()
print()
response = request.send().decode()
print(response)


GET / HTTP/1.1
Host: example.com
Connection: close

anything

HTTP/1.1 200 OK
Content-Type: text/html
ETag: "bc2473a18e003bdb249eba5ce893033f:1760028122.592274"
Last-Modified: Thu, 09 Oct 2025 16:42:02 GMT
Cache-Control: max-age=86000
Date: Fri, 14 Nov 2025 23:05:34 GMT
Content-Length: 513
Connection: close
X-N: S

<!doctype html><html lang="en"><head><title>Example Domain</title><meta name="viewport" content="width=device-width, initial-scale=1"><style>body{background:#eee;width:60vw;margin:15vh auto;font-family:system-ui,sans-serif}h1{font-size:1.5em}div{opacity:0.8}a:link,a:visited{color:#348}</style><body><div><h1>Example Domain</h1><p>This domain is for use in documentation examples without needing permission. Avoid use in operations.<p><a href="https://iana.org/domains/example">Learn more</a></div></body></html>

